## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,40.62,75,0,3.44,clear sky
1,1,Smithers,CA,54.7804,-127.1743,37.72,100,0,0.00,mist
2,2,Saldanha,ZA,-33.0117,17.9442,56.98,72,0,5.23,clear sky
3,3,Rikitea,PF,-23.1203,-134.9692,73.08,75,0,13.91,clear sky
4,4,Cherskiy,RU,68.7500,161.3000,40.91,43,92,12.71,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,73.08,75,0,13.91,clear sky
5,5,Hobart,AU,-42.8794,147.3294,60.66,65,0,5.75,clear sky
8,8,Aguimes,ES,27.9054,-15.4461,68.63,83,20,6.91,few clouds
9,9,Busselton,AU,-33.6500,115.3333,63.21,67,5,17.34,clear sky
10,10,Baringo,KE,0.4667,35.9667,79.32,45,0,5.19,clear sky
11,11,Portland,US,45.5234,-122.6762,67.78,69,0,0.00,clear sky
13,13,Caravelas,BR,-17.7125,-39.2481,66.00,84,100,2.64,overcast clouds
16,16,Arraial Do Cabo,BR,-22.9661,-42.0278,67.42,80,13,23.31,few clouds
17,17,Salmon Creek,US,45.7107,-122.6490,67.32,69,0,0.00,clear sky
18,18,Butaritari,KI,3.0707,172.7902,82.40,68,100,7.99,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,73.08,75,0,13.91,clear sky
5,5,Hobart,AU,-42.8794,147.3294,60.66,65,0,5.75,clear sky
8,8,Aguimes,ES,27.9054,-15.4461,68.63,83,20,6.91,few clouds
9,9,Busselton,AU,-33.6500,115.3333,63.21,67,5,17.34,clear sky
10,10,Baringo,KE,0.4667,35.9667,79.32,45,0,5.19,clear sky
11,11,Portland,US,45.5234,-122.6762,67.78,69,0,0.00,clear sky
13,13,Caravelas,BR,-17.7125,-39.2481,66.00,84,100,2.64,overcast clouds
16,16,Arraial Do Cabo,BR,-22.9661,-42.0278,67.42,80,13,23.31,few clouds
17,17,Salmon Creek,US,45.7107,-122.6490,67.32,69,0,0.00,clear sky
18,18,Butaritari,KI,3.0707,172.7902,82.40,68,100,7.99,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,73.08,clear sky,-23.1203,-134.9692,
5,Hobart,AU,60.66,clear sky,-42.8794,147.3294,
8,Aguimes,ES,68.63,few clouds,27.9054,-15.4461,
9,Busselton,AU,63.21,clear sky,-33.6500,115.3333,
10,Baringo,KE,79.32,clear sky,0.4667,35.9667,
11,Portland,US,67.78,clear sky,45.5234,-122.6762,
13,Caravelas,BR,66.00,overcast clouds,-17.7125,-39.2481,
16,Arraial Do Cabo,BR,67.42,few clouds,-22.9661,-42.0278,
17,Salmon Creek,US,67.32,clear sky,45.7107,-122.6490,
18,Butaritari,KI,82.40,overcast clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...sk

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,73.08,clear sky,-23.1203,-134.9692,People ThankYou
5,Hobart,AU,60.66,clear sky,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
8,Aguimes,ES,68.63,few clouds,27.9054,-15.4461,Hotel Villa de Aguimes
9,Busselton,AU,63.21,clear sky,-33.6500,115.3333,Observatory Guest House
10,Baringo,KE,79.32,clear sky,0.4667,35.9667,Khoisang Hostels


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
